In [ ]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.3.1'
spark_version = 'spark-3.4.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:5 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:8 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease [18.1 kB]
Get:9 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Get:11 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packages [1,057 kB]
Hit:12 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Get:13 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 Packages [3,253 kB]
Get:14

In [ ]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("CSV Import").getOrCreate()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
csv_path2 = "/content/drive/MyDrive/notbalanced"

In [ ]:
dfnb= spark.read.csv(csv_path2, header=True, inferSchema=True)

In [ ]:
dfnb.printSchema()

root
 |-- Diabetes_binary: double (nullable = true)
 |-- HighBP: double (nullable = true)
 |-- HighChol: double (nullable = true)
 |-- CholCheck: double (nullable = true)
 |-- BMI: double (nullable = true)
 |-- Smoker: double (nullable = true)
 |-- Stroke: double (nullable = true)
 |-- HeartDiseaseorAttack: double (nullable = true)
 |-- PhysActivity: double (nullable = true)
 |-- Fruits: double (nullable = true)
 |-- Veggies: double (nullable = true)
 |-- HvyAlcoholConsump: double (nullable = true)
 |-- AnyHealthcare: double (nullable = true)
 |-- NoDocbcCost: double (nullable = true)
 |-- GenHlth: double (nullable = true)
 |-- MentHlth: double (nullable = true)
 |-- PhysHlth: double (nullable = true)
 |-- DiffWalk: double (nullable = true)
 |-- Sex: double (nullable = true)
 |-- Age: double (nullable = true)
 |-- Education: double (nullable = true)
 |-- Income: double (nullable = true)



In [ ]:
dfnb.createOrReplaceTempView("diabetesnm")

In [ ]:
columns = dfnb.columns

In [ ]:
for column in columns:
    result = spark.sql(f"SELECT {column}, COUNT(*) AS count FROM diabetesnm GROUP BY {column}")
    result.show()


+---------------+------+
|Diabetes_binary| count|
+---------------+------+
|            0.0|218334|
|            1.0| 35346|
+---------------+------+

+------+------+
|HighBP| count|
+------+------+
|   0.0|144851|
|   1.0|108829|
+------+------+

+--------+------+
|HighChol| count|
+--------+------+
|     0.0|146089|
|     1.0|107591|
+--------+------+

+---------+------+
|CholCheck| count|
+---------+------+
|      0.0|  9470|
|      1.0|244210|
+---------+------+

+----+-----+
| BMI|count|
+----+-----+
|67.0|   15|
|70.0|   15|
|69.0|    9|
|88.0|    2|
|49.0|  416|
|98.0|    7|
|29.0|14890|
|64.0|   24|
|75.0|   52|
|47.0|  622|
|42.0| 1639|
|44.0| 1043|
|35.0| 5575|
|62.0|   43|
|96.0|    1|
|18.0| 1803|
|86.0|    1|
|39.0| 2911|
|34.0| 7181|
|37.0| 4147|
+----+-----+
only showing top 20 rows

+------+------+
|Smoker| count|
+------+------+
|   0.0|141257|
|   1.0|112423|
+------+------+

+------+------+
|Stroke| count|
+------+------+
|   0.0|243388|
|   1.0| 10292|
+------+-----

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import sklearn as skl
import matplotlib as plt
import tensorflow as tf


In [ ]:
df1nb = spark.sql("SELECT * FROM diabetesnm")
dfdbpandas=df1nb.toPandas()
dfdbpandas

,Diabetes_binary,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,...,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
0,0.0,1.0,1.0,1.0,40.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,5.0,18.0,15.0,1.0,0.0,9.0,4.0,3.0
1,0.0,0.0,0.0,0.0,25.0,1.0,0.0,0.0,1.0,0.0,...,0.0,1.0,3.0,0.0,0.0,0.0,0.0,7.0,6.0,1.0
2,0.0,1.0,1.0,1.0,28.0,0.0,0.0,0.0,0.0,1.0,...,1.0,1.0,5.0,30.0,30.0,1.0,0.0,9.0,4.0,8.0
3,0.0,1.0,0.0,1.0,27.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,0.0,0.0,0.0,0.0,11.0,3.0,6.0
4,0.0,1.0,1.0,1.0,24.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,3.0,0.0,0.0,0.0,11.0,5.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
253675,0.0,1.0,1.0,1.0,45.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,3.0,0.0,5.0,0.0,1.0,5.0,6.0,7.0
253676,1.0,1.0,1.0,1.0,18.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,4.0,0.0,0.0,1.0,0.0,11.0,2.0,4.0
253677,0.0,0.0,0.0,1.0,28.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,5.0,2.0
253678,0.0,1.0,0.0,1.0,23.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,3.0,0.0,0.0,0.0,1.0,7.0,5.0,1.0


In [ ]:
type(df1nb)

pyspark.sql.dataframe.DataFrame

In [ ]:
# Separate the data into labels and features

# Separate the y variable, the labels
y=dfdbpandas['Diabetes_binary']

# Separate the X variable, the features
X=dfdbpandas.drop('Diabetes_binary',axis=1)



In [ ]:
# Review the X variable DataFrame
X.head()

,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,Veggies,...,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
0,1.0,1.0,1.0,40.0,1.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,5.0,18.0,15.0,1.0,0.0,9.0,4.0,3.0
1,0.0,0.0,0.0,25.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,3.0,0.0,0.0,0.0,0.0,7.0,6.0,1.0
2,1.0,1.0,1.0,28.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,1.0,5.0,30.0,30.0,1.0,0.0,9.0,4.0,8.0
3,1.0,0.0,1.0,27.0,0.0,0.0,0.0,1.0,1.0,1.0,...,1.0,0.0,2.0,0.0,0.0,0.0,0.0,11.0,3.0,6.0
4,1.0,1.0,1.0,24.0,0.0,0.0,0.0,1.0,1.0,1.0,...,1.0,0.0,2.0,3.0,0.0,0.0,0.0,11.0,5.0,4.0


In [ ]:
# Review the y variable DataFrame
y.head()

0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: Diabetes_binary, dtype: float64

In [ ]:
# Check the balance of our target values
y.value_counts()

0.0    218334
1.0     35346
Name: Diabetes_binary, dtype: int64

In [ ]:
# Import the train_test_learn module
from sklearn.model_selection import train_test_split

# Split the data using train_test_split
# Assign a random_state of 1 to the function
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [ ]:
# Import the LogisticRegression module from SKLearn
from sklearn.linear_model import LogisticRegression

# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
logistic_regression_model = LogisticRegression(random_state=1)

# Fit the model using training data
lr_model = logistic_regression_model.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Since this option didn't work - used standard scaling and/or increasing the max iterations

In [ ]:
from sklearn.preprocessing import StandardScaler

# Instantiate the StandardScaler
scaler = StandardScaler()

# Scale the training data
X_train_scaled = scaler.fit_transform(X_train)

# Fit the model using the scaled training data
lr_model = logistic_regression_model.fit(X_train_scaled, y_train)

In [ ]:
# # Instantiate the Logistic Regression model with increased max_iter
logistic_regression_model = LogisticRegression(random_state=1, max_iter=1000)

# # Fit the model using training data
lr_model = logistic_regression_model.fit(X_train, y_train)

In [ ]:
# Make a prediction using the testing data
predicted_y_testvalues = logistic_regression_model.predict(X_test)
predicted_y_testvalues

array([0., 0., 0., ..., 0., 0., 0.])

In [ ]:
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report
# Print the balanced_accuracy score of the model
balanced_accuracy_score(y_test,predicted_y_testvalues)

0.5640019751109571

In [ ]:
# Generate a confusion matrix for the model
confusion_matrix(y_test,predicted_y_testvalues)

array([[53384,  1167],
       [ 7544,  1325]])

In [ ]:
# Print the classification report for the model
print(classification_report(y_test,predicted_y_testvalues))

              precision    recall  f1-score   support

         0.0       0.88      0.98      0.92     54551
         1.0       0.53      0.15      0.23      8869

    accuracy                           0.86     63420
   macro avg       0.70      0.56      0.58     63420
weighted avg       0.83      0.86      0.83     63420



Looking at the metrics for each class:

For class 0.0:

Precision: 0.88
Recall: 0.98
F1-score: 0.92
Support: 54551
For class 1.0:

Precision: 0.53
Recall: 0.15
F1-score: 0.23
Support: 8869

Overall Accuracy: 0.86

The macro average F1-score is 0.58, which represents the average F1-score across both classes. The weighted average F1-score is 0.83, which takes into account the class imbalance by weighting the F1-score by the number of samples in each class.

These metrics provide insights into the performance of your classification model. It appears that the model performs well in predicting class 0.0 (with high precision and recall), but it struggles with class 1.0, as indicated by the low precision, recall, and F1-score. The accuracy of 0.86 suggests that the model is correct in its predictions for 86% of the samples.

In [ ]:
# Import the RandomOverSampler module form imbalanced-learn
from imblearn.over_sampling import RandomOverSampler

# Instantiate the random oversampler model
# # Assign a random_state parameter of 1 to the model
oversampler=RandomOverSampler(random_state=1)


# Fit the original training data to the random_oversampler model
X_over, y_over = oversampler.fit_resample(X_test, y_test)

In [ ]:
# Count the distinct values of the resampled labels data
y_over.value_counts()

0.0    54551
1.0    54551
Name: Diabetes_binary, dtype: int64

In [ ]:
# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_over_scaled = scaler.fit_transform(X_over)

logistic_regression_model = LogisticRegression(random_state=1, solver='lbfgs')
lr_model = logistic_regression_model.fit(X_over_scaled, y_over)

X_scaler = scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)

In [ ]:
logistic_regression_model = LogisticRegression(random_state=1, max_iter=1000)
lr_model = logistic_regression_model.fit(X_over, y_over)

In [ ]:
# Make a prediction using the testing data
predicted_y_values = logistic_regression_model.predict(X_test)
predicted_y_values

array([0., 0., 1., ..., 1., 1., 0.])

In [ ]:
# Print the classification report for the model
print(classification_report(y_test, predicted_y_values))

              precision    recall  f1-score   support

         0.0       0.95      0.73      0.82     54551
         1.0       0.31      0.77      0.44      8869

    accuracy                           0.73     63420
   macro avg       0.63      0.75      0.63     63420
weighted avg       0.86      0.73      0.77     63420






These metrics indicate that the model has a relatively high precision for class 0 (negative class) but a low precision for class 1 (positive class). It has a higher recall for class 1 compared to class 0, indicating that it is better at identifying the positive class. The F1-score for class 0 is relatively high, while the F1-score for class 1 is lower, suggesting that the model's performance is better for the negative class.

The overall accuracy of the model is 0.73, which means it correctly predicts the class for approximately 73% of the instances. The macro average metrics provide an average measure across both classes, while the weighted average metrics take into account the class imbalance in the dataset.

